In [90]:
! pip install langchain_community tiktoken langchain_ollama langchainhub chromadb langchain unstructured

  Using cached python_magic-0.4.27-py2.py3-none-any.whl.metadata (5.8 kB)
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ------------------------------------- 981.5/981.5 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 12.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/590.6 kB ? eta -:--:--
   --------------------------------------- 590.6/590.6 kB 20.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ------------------------------ --------- 2.9/3.8 MB 15.2 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 15.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -----------

In [ ]:
# import bs4
# from langchain_community.document_loaders  import WebBaseLoader

# Load Documents
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )

# docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
# import os
# from langchain_community.document_loaders import TextLoader

# curr_path = os.path.abspath(os.curdir)
# # docFileName = "gfx101.txt"
# docFileName = "gfx_pipeline.txt" #copied till "matrix math", from 3d math onwards ingestion pending
# docFilePath = os.path.join(curr_path,'docs',docFileName)

# loader=TextLoader(docFilePath)
# docs = loader.load()

# print(len(docs))
# # print(len(docs[0]))

1


In [41]:
import os
from langchain_community.document_loaders import DirectoryLoader

#load webgpu
curr_path = os.path.abspath(os.curdir)
directory_path = os.path.join(curr_path,'docs','webgpu')
print(directory_path)

loader = DirectoryLoader(directory_path)
docs= loader.load()
print(len(docs))

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


d:\Coding_Projects\ai\gfx_rag\docs\webgpu


libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider insta

48


In [56]:
length = 0
for doc in docs:
    length+=len(doc.page_content)
print(length)

1011285


In [57]:
modelName ="llama3.2"
modelName ="deepseek-r1:1.5b"

In [72]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 1000, chunk_overlap = 100)
splits = text_splitter.split_documents(docs)

print (len(splits))

389


In [73]:
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

embedding_model_name="nomic-embed-text"
vector_store = Chroma.from_documents(documents=splits,
                                     embedding= OllamaEmbeddings(model=embedding_model_name))

retriever = vector_store.as_retriever()

In [114]:
question = "What is Task Decomposition for LLM agents?"
question = "What happens if any of the projected point coordinates is not within this range, for instance, if x' equals -1.1?"
question = "Can The Number Of Times Light Bouces off the surface of objects be infinite?"
question = "What is indirect lighting"
question = "Why is global illumination difficult"
question = "what is 'MSAA'?"
question = "Write a code for a simple histogram"
question = "How to request limits"
question = "What are radians?"
question = "What is an orange?"
question = "What is your name?"
# question = input('question?: ')

In [115]:
# from langchain.prompts import ChatPromptTemplate
# # Multi Query: Different Perspectives
# template = """You are an AI language model assistant. Your task is to generate five 
# different versions of the given user question to retrieve relevant documents from a vector 
# database. By generating multiple perspectives on the user question, your goal is to help
# the user overcome some of the limitations of the distance-based similarity search. 
# Provide these alternative questions separated by newlines. Original question: {question}"""

# prompt_perspectives = ChatPromptTemplate.from_template(template)

# from langchain_core.output_parsers import StrOutputParser
# from langchain_ollama import ChatOllama

# generate_queries = (
#     prompt_perspectives
#     | ChatOllama(model=modelName,temperature=0)
#     | StrOutputParser()
#     | (lambda x: x.split('\n'))
# )


In [116]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion 
    | ChatOllama(model=modelName,temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [117]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
retrieved_docs = retrieval_chain_rag_fusion.invoke({"question": question})

In [118]:
# from langchain.load import dumps, loads

# def get_unique_union(documents: list[list]):
#     """ Unique union of retrieved docs """
#     # Flatten list of lists, and convert each Document to string
#     flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
#     # Get unique documents
#     unique_docs = list(set(flattened_docs))
#     # Return
#     return [loads(doc) for doc in unique_docs]

# retrieval_chain = generate_queries | retriever.map() | get_unique_union
# retrieved_docs = retrieval_chain.invoke({"question":question})
# len(retrieved_docs)


In [119]:
print(len(retrieved_docs))
print([doc.page_content for (doc,_) in retrieved_docs])


27
['...', 'Now that we have a buffer and 2 views we can set the data in the structure.\n\n```js\n\nconst kVelocityOffset = 0;\n\nconst kAccelerationOffset = 1;\n\nconst kFrameCountOffset = 2;\n\nourStructValuesAsF32[kVelocityOffset] = 1.2; ourStructValuesAsF32[kAccelerationOffset] = 3.4; ourStructValuesAsU32[kFrameCountOffset] = 56;    // an integer value ```\n\nLike many things in programming there are multiple ways we could set the data forOutStruct.TypedArrays have a constructor that takes various forms. For example\n\nnew Float32Array(12)\n\nThis version makes anewArrayBuffer, in this case of 12 * 4 bytes. It then creates theFloat32Arrayto view it.\n\nnew Float32Array([4, 5, 6])\n\nThis version makes anewArrayBuffer, in this case of 3 * 4 bytes. It then creates theFloat32Arrayto view it. And it sets the initial values to 4, 5, 6.\n\nNote you can also pass anotherTypedArray. For example\n\nnew Float32Array(someUint8ArrayOf6Values)will make anewArrayBufferof size 6 * 4, then create 

In [120]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [121]:
llm = ChatOllama(
    model=modelName,
    temperature=0.8
)

In [122]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retrieval_chain_rag_fusion, "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

out = rag_chain.invoke({"question":question})

In [123]:
print(out)

<think>
Okay, let me try to figure this out. The user has provided several HTML files and some JavaScript snippets. They're asking specifically about their JavaScript code snippet that defines a `SceneGraphNode` class.

First, I'll look at the given code. It's an object with constructor parameters for `name` and `source`. Inside the constructor, it sets this.name to name and this.source to source. Then there's an edge comment saying "Select a finger...", which I think might be a typo or example text.

I don't see any other methods in this class, so nothing stands out as special except maybe that they might have added additional methods later, but the snippet provided doesn't show those. 

The comment about selecting a finger and just rotating x seems like an example of how the node selection works, but again, no code here to implement that functionality.

So, based on what's given, this class simply holds `name` and `source` properties without any methods. It might have been part of a 